In [90]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('train.csv')

In [38]:
df.shape

(1460, 81)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [6]:
# To check how many columns have missing values
col_miss_val_list = df.columns[df.isnull().any()].tolist()

In [7]:
col_miss_val_list

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [8]:
df[col_miss_val_list].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [39]:
corr = df.corr()['SalePrice']
corr[np.argsort(corr, axis=0)[::-1]]

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

Let's look at these variables with missing values.

LotFrontage: Linear feet of street connected to property. It has quiet low correlation with the price so we can remove it or put mean instead of missing values.

Alley: Type of alley access to property. Many missing values, maybe these properties just don't have an alley access. I would prefer to remove it. 

MasVnrType/MasVnrArea: both have 8 values missing, probably they are the same ones. I'm going to set most frequent one.

Bsmt.... Variables: A number of variables in connection with the basement. About the same number of missing values. However, there are two basement-related variables without missing values "BsmtFinSF1" and "BsmtFinSF2" - let's look at those and then decide what to do with the missing values.

Electrical: Just one missing value - here just impute most frequent one.

FireplaceQu: I assume the properties with missing values just don't have a fireplace. There's also the variable Fireplaces (without missing values) - check this and then decide.

Garage ... Variables: 81 missing in these columns. However, there are some Garage-related variables without missing values: GarageCars, GarageArea - check these and then decide.

PoolQC - probably no pool - but check against PoolArea (which has no missing values).

Fence: Too many missing values - probably no fence, prefer to remove it.

MiscFeature: Too many missing values - probably no fence, prefer to remove it.

In [57]:
#Remove 'MiscFeature', 'Fence', 'Alley'
df_cleaned = df.drop(['MiscFeature', 'Fence', 'Alley'], axis=1)

In [42]:
df_cleaned.shape

(1460, 78)

In [91]:
#LotFrontage
df_cleaned['LotFrontage'] = df_cleaned['LotFrontage'].fillna(df_cleaned['LotFrontage'].mean())

In [52]:
#MasVnrType/MasVnrArea
df[['MasVnrType','MasVnrArea']][df['MasVnrType'].isnull()==True]

,MasVnrType,MasVnrArea
234,NaN,NaN
529,NaN,NaN
650,NaN,NaN
936,NaN,NaN
973,NaN,NaN
977,NaN,NaN
1243,NaN,NaN
1278,NaN,NaN


As we can see the missing values for MasVnrType and MasVnrArea variables are in the same rows.

In [59]:
df['MasVnrType'].value_counts()

None       864
BrkFace    445
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

'None' is the most frequent value, so I replace missing MasVnrType values with 'None' and  missing MasVnrArea values with 0.

In [58]:
df_cleaned.loc[df_cleaned['MasVnrType'].isnull(),'MasVnrType'] = 'None'

In [61]:
df_cleaned.loc[df_cleaned['MasVnrArea'].isnull(),'MasVnrArea'] = 0.0

In [64]:
#basement
df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2']][df['BsmtQual'].isnull()==True]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtFinSF1,BsmtFinSF2
17,NaN,NaN,NaN,NaN,NaN,0,0
39,NaN,NaN,NaN,NaN,NaN,0,0
90,NaN,NaN,NaN,NaN,NaN,0,0
102,NaN,NaN,NaN,NaN,NaN,0,0
156,NaN,NaN,NaN,NaN,NaN,0,0
182,NaN,NaN,NaN,NaN,NaN,0,0
259,NaN,NaN,NaN,NaN,NaN,0,0
342,NaN,NaN,NaN,NaN,NaN,0,0
362,NaN,NaN,NaN,NaN,NaN,0,0
371,NaN,NaN,NaN,NaN,NaN,0,0


NaN values are in the rows with no basement, so they should also can be set as 'None'.

In [66]:
bsmt_columns = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']
for col in bsmt_columns:
    df_cleaned.loc[df_cleaned[col].isnull(),col] = 'None'

In [76]:
#fireplace
df[['Fireplaces','FireplaceQu']][df['FireplaceQu'].isnull()==True]

,Fireplaces,FireplaceQu
0,0,NaN
5,0,NaN
10,0,NaN
12,0,NaN
15,0,NaN
17,0,NaN
18,0,NaN
19,0,NaN
26,0,NaN
29,0,NaN


As we can see from above 690 just don't have a fireplace.

In [75]:
df_cleaned.loc[df_cleaned['FireplaceQu'].isnull(),'FireplaceQu'] = 'None'

In [80]:
#garage
garage_columns = ['GarageType','GarageQual','GarageCond','GarageYrBlt','GarageFinish','GarageCars','GarageArea']
df[garage_columns][df['GarageQual'].isnull()==True]

,GarageType,GarageQual,GarageCond,GarageYrBlt,GarageFinish,GarageCars,GarageArea
39,NaN,NaN,NaN,NaN,NaN,0,0
48,NaN,NaN,NaN,NaN,NaN,0,0
78,NaN,NaN,NaN,NaN,NaN,0,0
88,NaN,NaN,NaN,NaN,NaN,0,0
89,NaN,NaN,NaN,NaN,NaN,0,0
99,NaN,NaN,NaN,NaN,NaN,0,0
108,NaN,NaN,NaN,NaN,NaN,0,0
125,NaN,NaN,NaN,NaN,NaN,0,0
127,NaN,NaN,NaN,NaN,NaN,0,0
140,NaN,NaN,NaN,NaN,NaN,0,0


In [82]:
for col in garage_columns:
    if df[col].dtype==np.object:
        df_cleaned.loc[df_cleaned[col].isnull(),col] = 'None'
    else:
        df_cleaned.loc[df_cleaned[col].isnull(),col] = 0

In [84]:
#pool
df['PoolArea'][df['PoolQC'].isnull()==True].describe()

count    1453.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: PoolArea, dtype: float64

If PoolQC is empty, PoolArea is 0.

In [86]:
df_cleaned.loc[df_cleaned['PoolQC'].isnull(),'PoolQC'] = 'None'

In [88]:
#electrical
df['Electrical'].value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [96]:
df_cleaned.loc[df_cleaned['Electrical'].isnull(),'Electrical'] = 'SBrkr'

Checking for missing values:

In [95]:
df_cleaned.isnull().sum().describe()

count    78.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64